In [1]:
(load "../../common")
(cl-common:hello)

(load "on-lisp")

(in-package :on-lisp)
(hello)

T

"Hello, Common!"

T

#<PACKAGE "ON-LISP">

"Hello, On Lisp!"


"Hello, Common!" 
"Hello, On Lisp!" 

# Defining Functions

In [2]:
(defun double (x)
  (* x 2))

DOUBLE

In [3]:
(double 1)

2

In [4]:
#'double

#<FUNCTION DOUBLE>

In [5]:
(function double)

#<FUNCTION DOUBLE>

In [6]:
(eq #'double (function double))

T

In [7]:
(eq #'double (car (list #'double)))

T

In [12]:
; lambda expression
; #'(lambda (x) (* x 2))
(format t "~A~%" #'(lambda (x) (* x 2)))

NIL

#<FUNCTION (LAMBDA (X)
  :IN
  "C:/Users/zhouj/AppData/Local/Temp/7EC2E24C70CC4E8CCF0FE027F8372D99-542064446.lisp") {1000D8E6CB}>


In [ ]:
(double 3)
((lambda (x) (* x 2)) 3) ; appear first in function call

6

6

In [14]:
; same function name and variable name
(let ((double 2))
  (double double))

4

In [17]:
; symbol-value
; symbol-function
(setq double 2)
(symbol-value 'double)
(symbol-function 'double)

2

2

#<FUNCTION DOUBLE>

SIMPLE-WARNING: undefined variable: ON-LISP::DOUBLE


In [18]:
; variable hold a function as its value
(setq x #'append)
(eq (symbol-value 'x) (symbol-function 'append))

#<FUNCTION APPEND>

T

SIMPLE-WARNING: undefined variable: ON-LISP::X


In [2]:
(double% 2)

4

# Functional Arguments

In [3]:
; apply, funcall
(+ 1 2)

(apply #'+ '(1 2))
(apply #'+ 1 '(2))
(apply (symbol-function '+) '(1 2))
(apply #'(lambda (x y) (+ x y)) '(1 2))

(funcall #'+ 1 2)

3

3

3

3

3

3

In [5]:
(mapcar #'(lambda (x) (+ x 10))
  '(1 2 3))
(mapcar #'+
    '(1 2 3)
  '(10 100 1000))

(sort '(1 4 2 5 6 7 3) #'<)
(remove-if #'evenp '(1 2 3 4 5 6 7))

(11 12 13)

(11 102 1003)

(1 2 3 4 5 6 7)

(1 3 5 7)

SB-INT:CONSTANT-MODIFIED: Destructive function SORT called on constant data: (1 4 2 5 6 7 3)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1


# Functions as Properties

In [1]:
(defun wag-tail () (princ 'wag-tail))
(defun bark () (princ 'bark))
(defun scurry () (princ 'scurry))
(defun squeak () (princ 'squeak))
(defun rub-legs () (princ 'rub-legs))
(defun scratch-carpet () (princ 'scratch-carpet))

(defun behave% (animal)
  (case animal
    (dog (wag-tail)
         (bark))
    (rat (scurry)
         (squeak))
    (cat (rub-legs)
         (scratch-carpet))))

(defun behave (animal)
  (funcall (get animal 'behavior)))

(setf (get 'dog 'behavior)
  #'(lambda ()
      (wag-tail)
      (bark)))

WAG-TAIL

BARK

SCURRY

SQUEAK

RUB-LEGS

SCRATCH-CARPET

BEHAVE%

BEHAVE

#<FUNCTION (LAMBDA ()
             :IN
             "C:/Users/zhouj/AppData/Local/Temp/E8CA5AFF4C0E46439C21409DEB4C8656-2676620219.lisp") {1000FD1A3B}>

In [3]:
(behave% 'dog)


BARK

WAG-TAILBARK

In [4]:
(behave 'dog)

BARK

WAG-TAILBARK

# Scope

In [ ]:
(let ((y 7))
  (defun scope-test (x)
    (list x y)))

(let ((y 5))
  (scope-test 3))

; (3 7)
; lexically scoped

SCOPE-TEST

(3 7)

SB-INT:SIMPLE-STYLE-WARNING: The variable Y is defined but never used.


# Closures

In [6]:
(defun list+ (lst n)
  (mapcar #'(lambda (x) (+ x n)) lst))

(list+ '(1 2 3) 10)

LIST+

(11 12 13)

In [7]:
(let ((counter 0))
  (defun new-id () (incf counter))
  (defun reset-id () (setq counter 0)))

RESET-ID

In [2]:
(setq add2 (make-adder 2)
  add10 (make-adder 10))

(funcall add2 5)
(funcall add10 3)

#<FUNCTION (LAMBDA (X) :IN MAKE-ADDER) {110214495B}>

7

13

SIMPLE-WARNING: undefined variable: ON-LISP::ADD10
SIMPLE-WARNING: undefined variable: ON-LISP::ADD2
SIMPLE-WARNING: undefined variable: ON-LISP::ADD2
SIMPLE-WARNING: undefined variable: ON-LISP::ADD10


In [1]:
; change internal state
(defun make-adderb (n)
  #'(lambda (x &optional change)
      (if change
          (setq n x)
          (+ x n))))

(let ((addx (make-adderb 1)))
  (list
   (funcall addx 3)
   (funcall addx 100 t)
   (funcall addx 3)))

MAKE-ADDERB

(4 100 103)

In [ ]:
; closures share same object
(defun make-dbms (db)
  (list
   #'(lambda (key)
       (cdr (assoc key db)))
   #'(lambda (key val)
       (push (cons key val) db)
       key)
   #'(lambda (key)
       (setf db (delete key db :key #'car))
       key)))

(defun lookup (key db)
  (funcall (car db) key))

(let ((cities (make-dbms '((boston . us) (paris . france)))))
  (list
   (funcall (car cities) 'boston)
   (funcall (second cities) 'lodon 'england)
   (funcall (car cities) 'lodon)
   (lookup 'paris cities)))

MAKE-DBMS

LOOKUP

(US LODON ENGLAND FRANCE)

# Local Functions

In [1]:
; labels

(labels ((inc (x) (1+ x)))
  (inc 3))

4

In [3]:
; refer to other functions including self
(count-instances 'a '((a b c) (d a r p a) (d a r) (a a)))

(1 2 1 2)

# Tail Recursion

In [4]:
; not
(defun our-length (lst)
  (if (null lst)
      0
      (1+ (our-length (cdr lst)))))

; is
(defun our-find-if (fn lst)
  (if (funcall fn (car lst))
      (car lst)
      (our-find-if fn (cdr lst))))

OUR-LENGTH

OUR-FIND-IF

In [5]:
; use local function and accumulator to transform into tail-recursion
(defun our-length% (lst)
  (labels ((rec (lst acc)
                (if (null lst)
                    acc
                    (rec (cdr lst) (1+ acc)))))
    (rec lst 0)))

OUR-LENGTH%

In [ ]:
(our-length '(a b c d))
(our-length% '(a b c d))

4

4

In [7]:
; type declaration for compiler

(defun triangle (n)
  (labels ((tri (c n)
                (declare (type fixnum n c))
                (if (zerop n)
                    c
                    (tri (the fixnum (+ n c))
                         (the fixnum (- n 1))))))
    (tri 0 n)))

TRIANGLE

# Compilation

In [ ]:
(defun foo (x) (1+ x))

(compiled-function-p #'foo)
(compile 'foo)
(compiled-function-p #'foo)

(compile nil '(lambda (x) (+ x 2)))

; define
(progn (compile 'bar '(lambda (x) (* x 3)))
       (compiled-function-p #'bar))

FOO

T

FOO

NIL

NIL

...

#<FUNCTION (LAMBDA (X)
             :IN
             "C:/Users/zhouj/AppData/Local/Temp/FE9B18CD07F946BDA6FE346210D09645-437650464.lisp") {1000F7677B}>

NIL

NIL

T

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::BAR


In [1]:
(let ((y 2))
  (defun foo (x) (+ x y)))

(compile 'foo)

FOO

FOO

NIL

NIL

In [ ]:
(compile 'make-adder)

(compiled-function-p (make-adder 2)) ; return compiled functions

MAKE-ADDER

NIL

NIL

T

In [2]:
(defun foo (list) (+ (50th lst) 1))

(disassemble #'50th)
(disassemble #'foo)

FOO

NIL

NIL

SB-INT:SIMPLE-STYLE-WARNING: The variable LIST is defined but never used.
SIMPLE-WARNING: undefined variable: ON-LISP::LST

; disassembly for 50TH
; Size: 78 bytes. Origin: #x10010747EB    ; 50TH
;   7EB:  498B4510         MOV  RAX, [R13+16]   ; thread.binding-stack-pointer


;   7EF:  488945F8         MOV  [RBP-8], RAX

;   7F3:  41844424F8       TEST  AL, [R12-8]   ; safepoint


;   7F8:  4883EC10         SUB  RSP, 16

;   7FC:  488B15ADFFFFFF   MOV  RDX, [RIP-83]   ; '(INLINE 50TH)


;   803:  B902000000       MOV  ECX, 2

;   808:  48892C24         MOV  [RSP], RBP

;   80C:  488BEC           MOV  RBP, RSP

;   80F:  498B45F0         MOV  RAX, [R13-16]   ; thread.linkage-table


;   813:  FF9020280000     CALL  [RAX+10272]   ; PROCLAIM


;   819:  4C8B45F0         MOV  R8, [RBP-16]

;   81D:  BA62000000       MOV  EDX, 98

;   822:  498BF8           MOV  RDI, R8

;   825:  B904000000       MOV  ECX, 4

;   82A:  FF7508           PUSH  QWORD PTR [RBP+8]

;   82D:  498B45F0         MO